# EDA - Log Files

This Notebook is to analyze the Data after loading into the main Staging Tables

The Main Objectives are
- Defining the best Data types for all the Columns 
- Checking the Quality of the Data to handle corrupted data
- Defining the PKs, Distribution & Sorting Keys for optimization

In [1]:
import configparser
import psycopg2
import pandas as pd
from IAC_create_redshift_cluster import func_connect_to_redshift

In [2]:
try:
    # Connecting to the Cluster
    conn, cur = func_connect_to_redshift('dwh.cfg')

    dict_conn_info = conn.get_dsn_parameters()

    print(" Connected to {}, Host: {}, User: {}".format \
            (
            dict_conn_info.get('dbname')
            , dict_conn_info.get('host')
            , dict_conn_info.get('user')
        )
    )

except Exception as e:
    print(" Faild to Connect to the Cluster, {}".format(e))
    sys.exit(-1)

 Connected to sparkify_db, Host: sparkify-dwh.ct9qgawfx2gi.us-west-2.redshift.amazonaws.com, User: sparkify_user


In [3]:
conn

<connection object at 0x7f0a888fb048; dsn: 'user=sparkify_user password=xxx dbname=sparkify_db host=sparkify-dwh.ct9qgawfx2gi.us-west-2.redshift.amazonaws.com port=5439', closed: 0>

In [4]:
config = configparser.ConfigParser()
config.read('dwh.cfg')
config_dwh_arn = config.get('CLUSTER','dwh_arn')

## Log Data Loading in the Staging Table

In [5]:
cur.execute('CREATE SCHEMA IF NOT EXISTS STAGING_SCHEMA;')

In [6]:
cur.execute("""CREATE TABLE IF NOT EXISTS STAGING_SCHEMA.STG_LOG
(
    artist VARCHAR(250),
    auth VARCHAR(50),
    firstname VARCHAR(50),
    gender CHAR(1),
    iteminsession VARCHAR(10),
    lastname VARCHAR(50),
    length VARCHAR(50),
    level VARCHAR(10),
    location VARCHAR(100),
    method VARCHAR(10),
    page VARCHAR(50),
    registration VARCHAR(50),
    sessionid VARCHAR(10),
    song VARCHAR(250),
    status VARCHAR(10),
    ts VARCHAR(50),
    useragent VARCHAR(250),
    userid VARCHAR(10)
);
""")

In [7]:
conn.commit()

### Loading the data in the Staging Table

In [8]:
cur.execute("""
COPY STAGING_SCHEMA.STG_LOG 
from 's3://udacity-dend/log_data/' 
CREDENTIALS 'aws_iam_role={}'
JSON 's3://udacity-dend/log_json_path.json' REGION 'us-west-2';
""".format(config_dwh_arn))

In [9]:
conn.commit()

In [10]:
cur.execute("""SELECT COUNT(*) FROM STAGING_SCHEMA.STG_LOG""")

In [11]:
cur.fetchall()

[(8056,)]

In [12]:
cur.execute("""SELECT * FROM STAGING_SCHEMA.STG_LOG""")

In [13]:
# data is not huge and can fit in pandas Dataframe for Profiling
df_log_data = pd.DataFrame(data=cur.fetchall(),columns=['artist','auth','firstname','gender','iteminsession','lastname','length','level','location','method','page','registration','sessionid','song','status','ts','useragent','userid'])

## Data Profiling

In [14]:
df_log_data.head()

,artist,auth,firstname,gender,iteminsession,lastname,length,level,location,method,page,registration,sessionid,song,status,ts,useragent,userid
0,None,Logged Out,None,None,0,None,None,free,None,PUT,Login,None,52,None,307,1541207073796,None,
1,None,Logged In,Celeste,F,1,Williams,None,free,"Klamath Falls, OR",GET,Home,1541077528796,52,None,200,1541207123796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",53
2,Mynt,Logged In,Celeste,F,2,Williams,166.94811999999999,free,"Klamath Falls, OR",PUT,NextSong,1541077528796,52,Playa Haters,200,1541207150796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",53
3,Taylor Swift,Logged In,Celeste,F,3,Williams,230.47791000000001,free,"Klamath Falls, OR",PUT,NextSong,1541077528796,52,You Belong With Me,200,1541207316796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",53
4,Amy Winehouse,Logged In,Celeste,F,4,Williams,229.85097999999999,free,"Klamath Falls, OR",PUT,NextSong,1541077528796,52,Valerie,200,1541207546796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",53


In [15]:
df_log_data.describe(include='O')

,artist,auth,firstname,gender,iteminsession,lastname,length,level,location,method,page,registration,sessionid,song,status,ts,useragent,userid
count,6820,8056,7770,7770,8056,7770,6820,8056,7770,8056,8056,7770,8056,6820,8056,8056,7770,8056
unique,3148,2,85,2,128,87,3994,2,63,2,13,97,941,5189,3,8023,40,98
top,Coldplay,Logged In,Chloe,F,0,Cuevas,239.3073,paid,"San Francisco-Oakland-Hayward, CA",PUT,NextSong,1540940782796,589,You're The One,200,1541345129796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",49
freq,58,7770,791,5482,941,772,42,6291,776,7021,6820,772,128,37,7846,2,1098,772


### Checking Data Completness

In [16]:
df_log_data.isnull().sum()

artist           1236
auth                0
firstname         286
gender            286
iteminsession       0
lastname          286
length           1236
level               0
location          286
method              0
page                0
registration      286
sessionid           0
song             1236
status              0
ts                  0
useragent         286
userid              0
dtype: int64

In [17]:
# Checking the empty Strings in all the Columns
df_log_data.applymap(lambda x:x=="").any()

artist           False
auth             False
firstname        False
gender           False
iteminsession    False
lastname         False
length           False
level            False
location         False
method           False
page             False
registration     False
sessionid        False
song             False
status           False
ts               False
useragent        False
userid            True
dtype: bool

In [18]:
len(df_log_data[df_log_data.userid == ""])

286

From the Cells above we can observe the below :
- No Unique Column to be referenced for the records
- Data is not complete, as many Columns contain NULLs & Empty Strings
- NULLs & empty Strings Numbers are the same for many Columns which indicates a business logic behind these inconsistencies

## Loading Time Table Data

8023 Records should be inserted in Time table

In [19]:
cur.execute('CREATE SCHEMA IF NOT EXISTS SPARKIFY_SCHEMA;')

In [20]:
cur.execute("""
CREATE TABLE IF NOT EXISTS SPARKIFY_SCHEMA.TIME_TBL
(
TIME_ID BIGINT PRIMARY KEY,
TS_FORMATTED TIMESTAMP,
YEAR_COL INT,
MONTH_COL INT,
DAY_COL INT,
HOUR_COL INT,
DAY_NAME VARCHAR(50)
);
""")

In [21]:
conn.commit()

In [22]:
cur.execute("""
INSERT INTO SPARKIFY_SCHEMA.TIME_TBL
SELECT DISTINCT 
CAST(ts AS BIGINT) AS TIME_ID
,TIMESTAMP 'epoch' + ts/1000 *INTERVAL '1 second' AS TS_FORMATTED
,EXTRACT(year FROM (TIMESTAMP 'epoch' + ts/1000 *INTERVAL '1 second')) AS YEAR_COL
,EXTRACT(month FROM (TIMESTAMP 'epoch' + ts/1000 *INTERVAL '1 second')) AS MONTH_COL
,EXTRACT(day FROM (TIMESTAMP 'epoch' + ts/1000 *INTERVAL '1 second')) AS DAY_COL
,EXTRACT(hour FROM (TIMESTAMP 'epoch' + ts/1000 *INTERVAL '1 second')) AS HOUR_COL
,CASE 
WHEN EXTRACT(dayofweek FROM (TIMESTAMP 'epoch' + ts/1000 *INTERVAL '1 second')) = 0 THEN 'Sunday'
WHEN EXTRACT(dayofweek FROM (TIMESTAMP 'epoch' + ts/1000 *INTERVAL '1 second')) = 1 THEN 'Monday'
WHEN EXTRACT(dayofweek FROM (TIMESTAMP 'epoch' + ts/1000 *INTERVAL '1 second')) = 2 THEN 'Tuesday'
WHEN EXTRACT(dayofweek FROM (TIMESTAMP 'epoch' + ts/1000 *INTERVAL '1 second')) = 3 THEN 'Wednesday'
WHEN EXTRACT(dayofweek FROM (TIMESTAMP 'epoch' + ts/1000 *INTERVAL '1 second')) = 4 THEN 'Thursday'
WHEN EXTRACT(dayofweek FROM (TIMESTAMP 'epoch' + ts/1000 *INTERVAL '1 second')) = 5 THEN 'Friday'
WHEN EXTRACT(dayofweek FROM (TIMESTAMP 'epoch' + ts/1000 *INTERVAL '1 second')) = 6 THEN 'Saturday'
END AS DAY_NAME
FROM staging_schema.stg_log
LEFT OUTER JOIN SPARKIFY_SCHEMA.TIME_TBL
ON CAST(stg_log.ts AS BIGINT) = TIME_TBL.TIME_ID
WHERE TIME_TBL.TIME_ID IS NULL;
"""
)

In [23]:
conn.commit()

In [24]:
cur.execute("""SELECT COUNT(*) FROM SPARKIFY_SCHEMA.TIME_TBL;""")

In [25]:
cur.fetchall()

[(8023,)]

In [26]:
cur.execute("""SELECT * FROM SPARKIFY_SCHEMA.TIME_TBL;""")

In [27]:
list_time_tbl_cols = ['TIME_ID','TS_FORMATTED','YEAR_COL','MONTH_COL','DAY_COL','HOUR_COL','DAY_NAME']

In [28]:
df_time_tbl = pd.DataFrame(columns=list_time_tbl_cols,data=cur.fetchall())

In [29]:
df_time_tbl.head()

,TIME_ID,TS_FORMATTED,YEAR_COL,MONTH_COL,DAY_COL,HOUR_COL,DAY_NAME
0,1541239749796,2018-11-03 10:09:09.796,2018,11,3,10,Saturday
1,1541263189796,2018-11-03 16:39:49.796,2018,11,3,16,Saturday
2,1541264544796,2018-11-03 17:02:24.796,2018,11,3,17,Saturday
3,1541265446796,2018-11-03 17:17:26.796,2018,11,3,17,Saturday
4,1541267244796,2018-11-03 17:47:24.796,2018,11,3,17,Saturday


In [30]:
df_time_tbl.DAY_NAME.unique()

array(['Saturday', 'Friday', 'Sunday', 'Tuesday', 'Wednesday', 'Thursday',
       'Monday'], dtype=object)

In [31]:
df_time_tbl.YEAR_COL.unique()

array([2018])

In [32]:
df_time_tbl.MONTH_COL.unique()

array([11])

Data is just a sample dataset, which contains only 2018-11 Data

## Loading User Table Data

While Loading the Data of the User we need to respect the Order of the Data to have the Latest status for each user

From the First look the Number of Records that shuld be inserted into User Table is 98, **BUT This is Wrong**, they should be **97** because there are Empty String in this field

In [33]:
list_user_cols = ['userid','firstname','lastname','gender','level']

In [34]:
df_log_data[list_user_cols].head()

,userid,firstname,lastname,gender,level
0,,None,None,None,free
1,53,Celeste,Williams,F,free
2,53,Celeste,Williams,F,free
3,53,Celeste,Williams,F,free
4,53,Celeste,Williams,F,free


In [35]:
df_log_data[list_user_cols].isnull().sum()

userid         0
firstname    286
lastname     286
gender       286
level          0
dtype: int64

In [36]:
df_log_data[list_user_cols].applymap(lambda x:x=="").any()

userid        True
firstname    False
lastname     False
gender       False
level        False
dtype: bool

In [37]:
df_log_data[(df_log_data.auth =='Logged In')][list_user_cols].isnull().sum()

userid       0
firstname    0
lastname     0
gender       0
level        0
dtype: int64

In [38]:
df_log_data[(df_log_data.auth =='Logged In')][list_user_cols].applymap(lambda x:x=="").any()

userid       False
firstname    False
lastname     False
gender       False
level        False
dtype: bool

From the Above cells we have to filter the User data when the 'auth' column = 'Logged In' to get user Data.

In [39]:
# No. of users to be loaded
len(df_log_data[(df_log_data.auth =='Logged In')].userid.unique())

97

In [40]:
# Checking the Lengthes of all the Columns to define the approperiate Length & Data Type
for col in df_log_data.dtypes[df_log_data.dtypes.values == 'object'][list_user_cols].index:
    print("{} : {}".format(col,max(df_log_data[col].str.len().values)))

userid : 3
firstname : nan
lastname : nan
gender : nan
level : 4


In [41]:
# checking if all Users Ids are Numeric
df_log_data[(~df_log_data.userid.str.isdigit()) & (df_log_data.auth =='Logged In')][list_user_cols]

,userid,firstname,lastname,gender,level


In [42]:
cur.execute("""
CREATE TABLE IF NOT EXISTS SPARKIFY_SCHEMA.USER_TBL
(
USER_ID INTEGER PRIMARY KEY,
FIRST_NAME VARCHAR(50),
LAST_NAME VARCHAR(50),
GENDER CHAR(1),
LEVEL VARCHAR(10)
);
"""
)

In [43]:
conn.commit()

In [44]:
cur.execute("""
DELETE FROM SPARKIFY_SCHEMA.USER_TBL
USING staging_schema.stg_log
WHERE USER_TBL.USER_ID = stg_log.userid;
""")

In [45]:
conn.commit()

In [46]:
cur.execute("""
INSERT INTO SPARKIFY_SCHEMA.USER_TBL
SELECT USER_ID,FIRST_NAME,LAST_NAME,gender,level
FROM 
(
SELECT
CAST(userid AS INTEGER) AS USER_ID,
firstname AS FIRST_NAME,
lastname AS LAST_NAME,
gender,
level,
ROW_NUMBER() OVER (PARTITION BY userid order BY ts DESC) AS ROW_NUM
FROM staging_schema.stg_log
WHERE auth='Logged In'
) WHERE ROW_NUM = 1;
""")

In [47]:
conn.commit()

In [48]:
cur.execute("""SELECT * FROM SPARKIFY_SCHEMA.USER_TBL""")

In [49]:
df_user_data = pd.DataFrame(data=cur.fetchall(),columns=list_user_cols)

In [50]:
len(df_user_data)

97

In [51]:
df_user_data.head(10)

,userid,firstname,lastname,gender,level
0,12,Austin,Rosales,M,free
1,33,Bronson,Harris,M,free
2,51,Maia,Burke,F,free
3,63,Ayla,Johnson,F,free
4,74,Braden,Parker,M,free
5,98,Jordyn,Powell,F,free
6,20,Aiden,Ramirez,M,paid
7,28,Brantley,West,M,free
8,39,Walter,Frye,M,free
9,72,Hayden,Brock,F,paid


In [52]:
df_user_data.isnull().sum()

userid       0
firstname    0
lastname     0
gender       0
level        0
dtype: int64

## Loading Song Play Table

To Load Song Play Data, we have to filter 'page' col = 'NextSong'

In [53]:
df_log_data.columns

Index(['artist', 'auth', 'firstname', 'gender', 'iteminsession', 'lastname',
       'length', 'level', 'location', 'method', 'page', 'registration',
       'sessionid', 'song', 'status', 'ts', 'useragent', 'userid'],
      dtype='object')

In [54]:
list_song_play_cols = ['ts','sessionid','artist','song','userid','level','location','useragent','auth']

In [55]:
df_log_data[list_song_play_cols].head()

,ts,sessionid,artist,song,userid,level,location,useragent,auth
0,1541207073796,52,None,None,,free,None,None,Logged Out
1,1541207123796,52,None,None,53,free,"Klamath Falls, OR","""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",Logged In
2,1541207150796,52,Mynt,Playa Haters,53,free,"Klamath Falls, OR","""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",Logged In
3,1541207316796,52,Taylor Swift,You Belong With Me,53,free,"Klamath Falls, OR","""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",Logged In
4,1541207546796,52,Amy Winehouse,Valerie,53,free,"Klamath Falls, OR","""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",Logged In


### Checking Data Completness

In [56]:
df_log_data[list_song_play_cols].isnull().sum()

ts              0
sessionid       0
artist       1236
song         1236
userid          0
level           0
location      286
useragent     286
auth            0
dtype: int64

In [57]:
df_log_data[df_log_data.page == 'NextSong'][list_song_play_cols].isnull().sum()

ts           0
sessionid    0
artist       0
song         0
userid       0
level        0
location     0
useragent    0
auth         0
dtype: int64

In [58]:
df_log_data[df_log_data.page == 'NextSong'][list_song_play_cols].applymap(lambda x:x=="").any()

ts           False
sessionid    False
artist       False
song         False
userid       False
level        False
location     False
useragent    False
auth         False
dtype: bool

In [59]:
len(df_log_data[df_log_data.page == 'NextSong'])

6820

### Checking Data Uniquness

In [60]:
cur.execute(
"""
SELECT ts,sessionid,COUNT(*)
FROM STAGING_SCHEMA.STG_LOG
WHERE page = 'NextSong'
GROUP BY ts,sessionid
HAVING COUNT(*) > 1
""")

In [61]:
cur.fetchall()

[]

Then we will stick with current Columns as PK, ts & sessionid

### Number of records to be inserted in Song Play Table

In [62]:
cur.execute(
"""
SELECT COUNT(*)
FROM
(
SELECT ts,sessionid
FROM STAGING_SCHEMA.STG_LOG
WHERE page = 'NextSong'
GROUP BY ts,sessionid
)
""")

In [63]:
cur.fetchall()

[(6820,)]

6820 records shoud be inserted into Song Play Table

In [64]:
# Checking the Lengthes of all the Columns to define the approperiate Length & Data Type
for col in df_log_data.dtypes[df_log_data.dtypes.values == 'object'][list_song_play_cols].index:
    print("{} : {}".format(col,max(df_log_data[~(df_log_data[col].isnull())][col].str.len().values)))

ts : 13
sessionid : 4
artist : 89
song : 151
userid : 3
level : 4
location : 46
useragent : 139
auth : 10


In [65]:
# checking if all Users Ids are Numeric
df_log_data[(~df_log_data.sessionid .str.isdigit()) & (df_log_data.page == 'NextSong')][list_song_play_cols]

,ts,sessionid,artist,song,userid,level,location,useragent,auth


### Song Play Table DDL

In [66]:
cur.execute("""
CREATE TABLE IF NOT EXISTS SPARKIFY_SCHEMA.SONG_PLAY_TBL
(
UNIX_TS BIGINT,
SESSION_ID BIGINT,
ARTIST_ID VARCHAR(50) DEFAULT -9999,
SONG_ID VARCHAR(50) DEFAULT -9999,
USER_ID INTEGER DEFAULT -9999,
USER_LEVEL VARCHAR(10),
USER_LOCATION VARCHAR(100),
USER_AGENT VARCHAR(250),
USER_LOGGED VARCHAR(25),
CREATION_TIMESTAMP TIMESTAMP,
PRIMARY KEY (UNIX_TS,SESSION_ID)
);""")

In [67]:
conn.commit()

In [68]:
cur.execute("""
DELETE FROM SPARKIFY_SCHEMA.SONG_PLAY_TBL
USING staging_schema.stg_log
WHERE SONG_PLAY_TBL.UNIX_TS = stg_log.ts
AND SONG_PLAY_TBL.SESSION_ID = stg_log.sessionid
AND stg_log.page = 'NextSong';
""")

In [69]:
conn.commit()

### For Information why this Query please find the analysis below the SQL

In [70]:
cur.execute("""
INSERT INTO SPARKIFY_SCHEMA.SONG_PLAY_TBL
SELECT DISTINCT
CAST(ts AS BIGINT) AS UNIX_TS,
CAST(sessionid AS BIGINT) AS SESSION_ID,
ARTIST_OUTPUT.ARTIST_ID,
SONG_OUTPUT.SONG_ID,
CAST(userid AS INTEGER) AS USER_ID,
level,
location AS USER_LOCATION,
useragent AS USER_AGENT,
CAST(auth AS VARCHAR(25)) AS USER_LOGGED,
TIMESTAMP 'epoch' + ts/1000 *INTERVAL '1 second' AS CREATION_TIMESTAMP
FROM STAGING_SCHEMA.STG_LOG
LEFT JOIN 
(
    SELECT ARTIST_ID,ARTIST_NAME,
    ROW_NUMBER() OVER (PARTITION BY ARTIST_NAME ORDER BY ARTIST_LATITUDE DESC, ARTIST_LONGITUTE DESC,ARTIST_LOCATION ASC) AS ROW_NUM
    FROM SPARKIFY_SCHEMA.ARTIST_TBL
) AS ARTIST_OUTPUT
ON STG_LOG.artist = ARTIST_OUTPUT.ARTIST_NAME
AND ARTIST_OUTPUT.ROW_NUM = 1
LEFT JOIN
(
    SELECT SONG_ID,SONG_TITLE,
    ROW_NUMBER() OVER (PARTITION BY SONG_TITLE ORDER BY SONG_YEAR DESC) AS ROW_NUM
    FROM SPARKIFY_SCHEMA.SONG_TBL
) AS SONG_OUTPUT
ON TRIM(STG_LOG.song) = SONG_OUTPUT.SONG_TITLE
AND SONG_OUTPUT.ROW_NUM = 1
WHERE stg_log.page = 'NextSong'
""")

In [71]:
conn.commit()

In [72]:
cur.execute("""SELECT COUNT(*) FROM SPARKIFY_SCHEMA.SONG_PLAY_TBL""")

In [73]:
cur.fetchall()

[(6820,)]

### Anlyzing the Data to figure out how to solve the Duplication Problem, as there are Duplicates in Song Titles & Artists Names

In [74]:
# This is the final Query to get the wanted result to be inserted into the table
cur.execute("""
SELECT COUNT(*) FROM
(
SELECT DISTINCT
CAST(ts AS BIGINT) AS UNIX_TS,
CAST(sessionid AS BIGINT) AS SESSION_ID,
artist,
song,
CAST(userid AS INTEGER) AS USER_ID,
level,
location AS USER_LOCATION,
useragent AS USER_AGENT,
CAST(auth AS VARCHAR(25)) AS USER_LOGGED,
TIMESTAMP 'epoch' + ts/1000 *INTERVAL '1 second' AS CREATION_TIMESTAMP
FROM STAGING_SCHEMA.STG_LOG
LEFT JOIN 
(
    SELECT ARTIST_ID,ARTIST_NAME,
    ROW_NUMBER() OVER (PARTITION BY ARTIST_NAME ORDER BY ARTIST_LATITUDE DESC, ARTIST_LONGITUTE DESC,ARTIST_LOCATION ASC) AS ROW_NUM
    FROM SPARKIFY_SCHEMA.ARTIST_TBL
) AS ARTIST_OUTPUT
ON STG_LOG.artist = ARTIST_OUTPUT.ARTIST_NAME
AND ARTIST_OUTPUT.ROW_NUM = 1
LEFT JOIN
(
    SELECT SONG_ID,SONG_TITLE,
    ROW_NUMBER() OVER (PARTITION BY SONG_TITLE ORDER BY SONG_YEAR DESC) AS ROW_NUM
    FROM SPARKIFY_SCHEMA.SONG_TBL
) AS SONG_OUTPUT
ON TRIM(STG_LOG.song) = SONG_OUTPUT.SONG_TITLE
AND SONG_OUTPUT.ROW_NUM = 1
WHERE stg_log.page = 'NextSong'
) 
""")

In [75]:
cur.fetchall()

,0
0,6820


In [76]:
cur.execute("""
SELECT *,
ROW_NUMBER() OVER (PARTITION BY ARTIST_NAME ORDER BY ARTIST_LATITUDE DESC, ARTIST_LONGITUTE DESC,ARTIST_LOCATION ASC)
FROM SPARKIFY_SCHEMA.ARTIST_TBL
WHERE ARTIST_NAME IN
(
SELECT ARTIST_NAME
FROM SPARKIFY_SCHEMA.ARTIST_TBL
GROUP BY ARTIST_NAME
HAVING COUNT(*) > 1
)
ORDER BY ARTIST_NAME
""")

In [77]:
pd.DataFrame(cur.fetchall()).head(20)

,0,1,2,3,4,5
0,ARFMT4W1187FB42FA8,Alison Krauss,"Decatur, IL",-9999.00000,-9999.00000,1
1,ARF2SVO1187FB53E8F,Alison Krauss,Unknown,-9999.00000,-9999.00000,2
2,ARFQXJO11F50C4CFD4,Asheru And Blue Black Of The Unspoken Heard,Unknown,-9999.00000,-9999.00000,1
3,AR6OQLJ1187B9B032E,Asheru And Blue Black Of The Unspoken Heard,Unknown,-9999.00000,-9999.00000,2
4,AR4YYCG1187B9BA09E,Aterciopelados,Bogotá Colombia,4.65637,-74.11779,1
5,ARQDJVK11F33DFFFE5,Aterciopelados,Unknown,-9999.00000,-9999.00000,2
6,ARTDQRC1187FB4EFD4,Black Eyed Peas,"Los Angeles, CA",-9999.00000,-9999.00000,1
7,ARRR0C11187B98C59B,Black Eyed Peas,Unknown,-9999.00000,-9999.00000,2
8,ARG4K2W1187B99EA5D,Charles Aznavour,Virginia,38.00335,-79.77127,1
9,ARWJAMZ1187B9B32CA,Charles Aznavour,US,-9999.00000,-9999.00000,2


In [78]:
# A Query to get the Unique Artists Records based on their names, all duplicated names have different IDs, so they are still different and consistant with the DDL of the Artist tables
#, but to get the unique Artist for song play table we will get the records with the most available data in the other columns like ARTIST_LATITUDE DESC, ARTIST_LONGITUTE DESC,ARTIST_LOCATION ASC
cur.execute("""
SELECT COUNT(*) FROM
(
SELECT ARTIST_ID,ARTIST_NAME,
ROW_NUMBER() OVER (PARTITION BY ARTIST_NAME ORDER BY ARTIST_LATITUDE DESC, ARTIST_LONGITUTE DESC,ARTIST_LOCATION ASC) AS ROW_NUM
FROM SPARKIFY_SCHEMA.ARTIST_TBL
) AS ARTIST_OUTPUT
WHERE ROW_NUM = 1
""")

In [79]:
cur.fetchall()

[(9506,)]

In [80]:
cur.execute("""
SELECT * FROM SPARKIFY_SCHEMA.SONG_TBL
WHERE SONG_TITLE IN
(
SELECT SONG_TITLE
FROM SPARKIFY_SCHEMA.SONG_TBL
GROUP BY SONG_TITLE
HAVING COUNT(*) > 1
)
ORDER BY SONG_TITLE
""")

In [81]:
pd.DataFrame(data=cur.fetchall()).head(20)

,0,1,2,3,4
0,SOKOGIP12AB0182FCD,AROS1ML1187FB4CF35,200.90730,Adrenaline,2007
1,SODEQCE12A8C13EB88,ARWQU7G1187B99B7E6,298.86649,Adrenaline,2007
2,SOGWOQM12A8C13D5AE,AR6K9YV1187B9B4BF9,312.24118,Affliction,2008
3,SOISFRK12A6D4F573D,AR2FAMK1187FB4CF60,328.72444,Affliction,2006
4,SODADQS12AB018294F,ARGA92H1187B9B5E5D,155.74159,Africa,-9999
5,SOKWAIM12A8C137350,AR0693R1187FB59D32,173.97506,Africa,1998
6,SOQMKZE12A8C134EAB,ARZ9W8K1187B99793C,198.84363,Airport,2007
7,SODGJMJ12A8C140C98,AR655VL1187B9B61E3,274.23302,Airport,2005
8,SODVFGB12A8AE4615E,ARR1C0N1187FB372DF,187.24526,All I Need,1988
9,SOCMYPE12AC3DF7FDB,ARKIOZX122BCFCB4DC,193.27955,All I Need,2009


In [82]:
# doing the same for Song Table
cur.execute("""
SELECT SONG_ID,SONG_TITLE,SONG_YEAR,
ROW_NUMBER() OVER (PARTITION BY SONG_TITLE ORDER BY SONG_YEAR DESC) AS ROW_NUM
FROM SPARKIFY_SCHEMA.SONG_TBL
WHERE SONG_TITLE IN
(
SELECT SONG_TITLE
FROM SPARKIFY_SCHEMA.SONG_TBL
GROUP BY SONG_TITLE
HAVING COUNT(*) > 1
)
ORDER BY SONG_TITLE
""")

In [83]:
pd.DataFrame(data=cur.fetchall()).head(20)

,0,1,2,3
0,SODEQCE12A8C13EB88,Adrenaline,2007,1
1,SOKOGIP12AB0182FCD,Adrenaline,2007,2
2,SOGWOQM12A8C13D5AE,Affliction,2008,1
3,SOISFRK12A6D4F573D,Affliction,2006,2
4,SOKWAIM12A8C137350,Africa,1998,1
5,SODADQS12AB018294F,Africa,-9999,2
6,SOQMKZE12A8C134EAB,Airport,2007,1
7,SODGJMJ12A8C140C98,Airport,2005,2
8,SOCMYPE12AC3DF7FDB,All I Need,2009,1
9,SODVFGB12A8AE4615E,All I Need,1988,2
